# Analysing binned pseudorapidity

In [1]:
# System imports
import os
import sys
from pprint import pprint as pp
from time import time as tt
import pickle

# External imports
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import multiprocessing as mp
from functools import partial

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
import torch_geometric
from torch_geometric.data import Data

# Limit CPU usage on Jupyter
os.environ['OMP_NUM_THREADS'] = '4'

# Pick up local packages
sys.path.append('..')

# Locals

from prepareTracks import *
from nb_utils import *
from gpu_utils import *
from datasets import get_data_loaders
from trainers import get_trainer
import distributed

## Make File List

In [2]:
input_dir = '/global/cscratch1/sd/danieltm/ExaTrkX/trackml/train_100_events/'
output_dir = '/global/u2/d/danieltm/ExaTrkX/eta-tracker/data/numba_testing'
n_files = 32
n_workers = 32
config = {'selection': {'pt_min': 0.5,
    'phi_slope_max': 0.0006,
    'z0_max': 150,
    'n_phi_sections': 1,
    'n_eta_sections': 1,
    'eta_range': [-5, 5]}}

In [3]:
all_files = os.listdir(input_dir)
suffix = '-hits.csv'
file_prefixes = sorted(os.path.join(input_dir, f.replace(suffix, ''))
                       for f in all_files if f.endswith(suffix))
file_prefixes = np.array(file_prefixes[:n_files])

## CPU Threading (for comparison)

In [5]:
%%time
# Prepare output
os.makedirs(output_dir, exist_ok=True)
logging.info('Writing outputs to ' + output_dir)

# Process input files with a worker pool
with mp.Pool(processes=n_workers) as pool:
    process_func = partial(process_event, output_dir=output_dir,
                           phi_range=(-np.pi, np.pi), **config['selection'])
    pool.map(process_func, file_prefixes)

../prepareTracks.py:213: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(p_zr[0])
/global/homes/d/danieltm/.local/lib/python3.7/site-packages/scipy/optimize/minpack.py:449: RuntimeWarning: gtol=0.000000 is too small, func(x) is orthogonal to the columns of
  the Jacobian to machine precision.
  warnings.warn(errors[info][0], RuntimeWarning)
../prepareTracks.py:213: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(p_zr[0])
/global/homes/d/danieltm/.local/lib/python3.7/site-packages/scipy/optimize/minpack.py:449: RuntimeWarning: gtol=0.000000 is too small, func(x) is orthogonal to the columns of
  the Jacobian to machine precision.
  warnings.warn(errors[info][0], RuntimeWarning)
../prepareTracks.py:213: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(p_zr[0])
../prepareTracks.py:213: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(p_zr[0])
../prepareTracks.py:213: RuntimeWarning: invalid value

CPU times: user 131 ms, sys: 287 ms, total: 417 ms
Wall time: 1min 17s


## GPU Threading

## JIT Functions

In [4]:
@njit
def calc_dphi(phi1, phi2):
    """Computes phi2-phi1 given in range [-pi,pi]"""
    dphi = phi2 - phi1
    dphi[dphi > np.pi] -= 2*np.pi
    dphi[dphi < -np.pi] += 2*np.pi
    return dphi

@njit
def calc_eta(r, z):
    theta = np.arctan2(r, z)
    return -1. * np.log(np.tan(theta / 2.))

In [ ]:
import math
x = np.arange(10000000000).reshape(100000, 100000)

def go_slow(a): # Function is compiled to machine code when called the first time
    trace = 0.
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
#     return a + trace              # Numba likes NumPy broadcasting

@cuda.jit # Set "nopython" mode for best performance, equivalent to @njit
def go_fast_gpu(a): # Function is compiled to machine code when called the first time
    trace = float(0.)
    for i in range(a.shape[0]):   # Numba likes loops
        trace += float(math.tanh(float(a[i, i]))) # Numba likes NumPy functions
#     a = a+trace              # Numba likes NumPy broadcasting

@njit # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = float(0.)
    for i in range(a.shape[0]):   # Numba likes loops
        trace += float(math.tanh(float(a[i, i]))) # Numba likes NumPy functions
#     a = a+trace              # Numba likes NumPy broadcasting

In [ ]:
%%time
go_slow(x)

In [53]:
%%time
go_fast(x)

CPU times: user 20 µs, sys: 11 µs, total: 31 µs
Wall time: 37.2 µs


In [54]:
%%time
go_fast_gpu(x)

CPU times: user 316 ms, sys: 757 µs, total: 316 ms
Wall time: 315 ms


## Initial investigation

Set up the inital constraints and choose an event

Select end-cap volumes

In [28]:
# This is ONLY the end-cap
# vlids = [(9,2), (9,4), (9,6), (9,8), (9,10), (9,12), (9,14)]
# These are the other front detectors
#                     (14,2), (14,4), (14,6), (14,8), (14,10), (14,12),
#                     (18,2), (18,4), (18,6), (18,8), (18,10), (18,12)]
# These are the barrel volumes
vlids = [(8,2), (8,4), (8,6), (8,8),
             (13,2), (13,4), (13,6), (13,8),
             (17,2), (17,4)]
# eta_region = [0,0.2]

n_det_layers = len(vlids)
vlid_groups = hits.groupby(['volume_id', 'layer_id'])
hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer=i)
                      for i in range(n_det_layers)])

In [29]:
pt = np.sqrt(particles.px**2 + particles.py**2)
particles = particles[pt > pt_min]
particles = particles.join(pd.DataFrame(pt, columns=["pt"]))

In [30]:
truth = (truth[['hit_id', 'particle_id']]
             .merge(particles[['particle_id', 'nhits', 'pt']], on='particle_id'))

In [31]:
theta_hits = np.arccos(hits.z / (hits.x**2 + hits.y**2 + hits.z**2)**(0.5))
eta_hits = -np.log(np.tan(theta_hits/2))
phi_hits = np.arctan2(hits.y, hits.x)
r_hits = np.sqrt(hits.x**2 + hits.y**2)

In [32]:
# Create big hits + truth D.F.
hits = (hits[['hit_id', 'x', 'y', 'z', 'layer', 'evtid']]
            .assign(r=r_hits, phi=phi_hits, eta=eta_hits)
            .merge(truth[['hit_id', 'particle_id', 'nhits', 'pt']], on='hit_id'))
# Remove duplicated
hits = hits.loc[
        hits.groupby(['particle_id', 'layer'], as_index=False).r.idxmin()
    ]
hits = hits[hits['nhits'] > 2]

In [89]:
hits = hits.assign(**{'{}'.format(k):v for k,v in zip(["C", "D","E","F","G"], np.zeros(len(hits)))})

In [102]:
for pid in pd.unique(hits['particle_id']):
    hits.loc[hits.particle_id == pid, ["C", "D", "E", "F", "G"]] = get_track_parameters(hits[hits.particle_id == pid]['x'].to_numpy(), hits[hits.particle_id == pid]['y'].to_numpy(), hits[hits.particle_id == pid]['z'].to_numpy())

Select certain eta region

In [9]:
eta_region = [3.2,4]
hits = hits[(hits.eta > eta_region[0]) & (hits.eta < eta_region[1])]
# hits = hits.sort_values(by='eta')

Organise into adjacent layers

In [10]:
l = np.arange(n_det_layers)
layer_pairs = np.stack([l[:-1], l[1:]], axis=1)

In [11]:
feature_names = ['r', 'phi', 'z']
feature_scale = np.array([1000., np.pi / n_phi_sections, 1000.])

In [12]:
graph, IDs = construct_graph(hits, layer_pairs=layer_pairs,
                              phi_slope_max=phi_slope_max, z0_max=z0_max,
                              feature_names=feature_names,
                              feature_scale=feature_scale)

In [13]:
Ri_rows, Ri_cols = graph.Ri.nonzero()
Ro_rows, Ro_cols = graph.Ro.nonzero()
n_edges = Ri_cols.shape[0]
edge_index = np.zeros((2, n_edges), dtype=int)
edge_index[0, Ro_cols] = Ro_rows
edge_index[1, Ri_cols] = Ri_rows
print(len(graph.X), " hits and ", edge_index.shape[1], " edges")

634  hits and  524  edges


In [14]:
X, Y, edges, labels = graph.X, graph.y, edge_index, graph.y
X = X*feature_scale